# Ensembles: Comparación



Vamos a comparar el rendimiento de los siguientes algoritmos:

- Árboles de decisión
- Bagging sobre Árboles de decisión
- Random Forest
- Extra Trees

Para ello vamos a comenzar con la lectura del dataset de aceptabilidad de autos.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
names = ['buying','maint','doors','persons','lug_boot','safety','acceptability']
df = pd.read_csv(url,names=names)
df.dtypes

,0
buying,object
maint,object
doors,object
persons,object
lug_boot,object
safety,object
acceptability,object


In [2]:
df

,buying,maint,doors,persons,lug_boot,safety,acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   buying         1728 non-null   object
 1   maint          1728 non-null   object
 2   doors          1728 non-null   object
 3   persons        1728 non-null   object
 4   lug_boot       1728 non-null   object
 5   safety         1728 non-null   object
 6   acceptability  1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


Esta vez vamos a codificar los atributos usando un esquema One Hot, es decir, los consideraremos como variables categóricas. También vamos a codificar el target usando el `LabelEncoder`.

In [ ]:
# Codificación de etiquetas con LabelEncoder

# En este ejemplo se transforma una variable categórica en **números enteros**, lo cual es necesario para la mayoría de los algoritmos de scikit-learn.

from sklearn.preprocessing import LabelEncoder

# Crear el codificador
lab_enc = LabelEncoder()

# Ajustar el codificador a la columna 'acceptability'
# Esto asigna un número entero único a cada categoría
lab_enc.fit(df['acceptability'])

LabelEncoder()

In [ ]:
# Preparación de datos para Machine Learning

# En este bloque se generan las **variables independientes (X)** y la **variable objetivo (y)** a partir del DataFrame `df`.

# Variable objetivo codificada
y = lab_enc.transform(df['acceptability'])
# Convierte las etiquetas de 'acceptability' en números enteros
# Ejemplo: 'low' -> 0, 'medium' -> 1, 'high' -> 2

# Variables independientes: convertir categóricas en dummies
X = pd.get_dummies(df.drop('acceptability', axis=1))
# Elimina la columna objetivo y aplica one-hot encoding a las variables categóricas

# Mostrar las primeras filas de las primeras 8 columnas
X.iloc[:, 0:8].head()


,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh
0,False,False,False,True,False,False,False,True
1,False,False,False,True,False,False,False,True
2,False,False,False,True,False,False,False,True
3,False,False,False,True,False,False,False,True
4,False,False,False,True,False,False,False,True


In [5]:
X.shape

(1728, 21)

In [6]:
X.columns

Index(['buying_high', 'buying_low', 'buying_med', 'buying_vhigh', 'maint_high',
       'maint_low', 'maint_med', 'maint_vhigh', 'doors_2', 'doors_3',
       'doors_4', 'doors_5more', 'persons_2', 'persons_4', 'persons_more',
       'lug_boot_big', 'lug_boot_med', 'lug_boot_small', 'safety_high',
       'safety_low', 'safety_med'],
      dtype='object')

Hacemos el split entre train y test sets.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=41)

Para que los resultados sean consistentes hay que exponer los modelos exactamente al mismo esquema de validación cruzada.

In [ ]:
# Definición de validación cruzada estratificada

# En este bloque se prepara un esquema de **cross-validation** estratificada para evaluar modelos de clasificación.

from sklearn.model_selection import cross_val_score, StratifiedKFold

# Crear un objeto de validación cruzada estratificada
cv = StratifiedKFold(
    n_splits=3,       # Número de "folds" (subconjuntos) para dividir los datos
    random_state=41,  # Semilla para reproducibilidad
    shuffle=True      # Mezcla aleatoriamente los datos antes de dividirlos
)


##Comparando la performance de los árboles de decisión y ensambles de modelos

Ahora vamos a inicializar el clasificador de árbol de decisión, evaluar su rendimiento y compararlo con la perfomance de los ensambles que hemos visto hasta aquí. Para ello, vamos a usar los siguientes métodos:

### RandomForestClassifier()

Este método implementa y ejectua un RandomForest para resolver un problema de clasificación. Algunos de los parámetros más importantes son los siguientes:

* `n_estimators`: el número de iteraciones (o sea, de `base_estimators`) para entrenar
* `criterion`: define el criterio de impureza para evaluar la calidad de las particiones (por defecto, es `gini`)
* `max_features`: la cantidad de features que extraerá para entrenar cada `base_estimator`. Por default es igual a `sqrt(X.shape[1])`
* `bootstrap` y `bootstrap_features`: controla si tanto los n_samples como las features son extraidos con reposición.
* `max_depth`: la pronfundidad máxima del árbol
* `min_samples_leaf`: el número mínimo de n_samples para constituir una hoja del árbol (nodo terminal)
* `min_samples_split`: el número mínimo de n_samples para realizar un split.

y varios otros que pueden llegar a ser importantes al momento de realizar el tunning. En general, los más importantes suelen ser: `n_estimators`, `max_features`, `max_depth` y `min_samples_leaf`.


### ExtraTreesClassifier()

Con este método se puede estimar un conjunto de conjuntos de árboles de decisión randomizados. Toma los mismos parámetros que `RandomForestClassifier()`.


### BaggingClassifier()

Este método es muy interesante porque, a diferencia de los anteriores, es un "meta estimador", está situado en nivel de abstracción mayor. Es decir, que permite implementar el algoritmo de bagging (para clasificación) con casi cualquier estimador de Scikit-Learn. Toma como parámetros análogos a los dos métodos anteriores (con diferentes valores por defecto en algunos casos). Los únicos "nuevos" son:

* `base_estimator`: el estimador sobre el cual queremos correr el bagging (regresiones, árboles, etc...)
* `max_samples`: la cantidad de n_samples que muestrea en cada iteración. Por default es igual a `sqrt(X.shape[0])`


Para comparar los diferentes algoritmos armamos la siguiente función. Toma como input un estimador y un string con el nombre que le quieran poner, y ejecuta un `cross_val_score`

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier


def evaluar_rendimiento(modelo, nombre, X_train, y_train, cv):
    s = cross_val_score(modelo, X_train, y_train, cv=cv, n_jobs=-1)
    print("Rendimiento de {}:\t{:0.3} ± {:0.3}".format( \
        nombre, s.mean().round(3), s.std().round(3)))


Rendimiento de Árbol de decisión:	0.956 ± 0.008


Ahora probamos con los modelos restantes y evaluamos el rendimiento.  
 * Bagging de Árboles de decisión
 * RandomForest
 * ExtraTrees

Documentación.   
http://scikit-learn.org/stable/modules/ensemble.html#forest

In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced', random_state=1)
bdt = BaggingClassifier(dt, random_state=1)
rf = RandomForestClassifier(class_weight='balanced', random_state=1)
et = ExtraTreesClassifier(class_weight='balanced', random_state=1)

evaluar_rendimiento(dt,  "Árbol de decisión", X_train, y_train, cv)
evaluar_rendimiento(bdt, "Bagging AD", X_train, y_train, cv)
evaluar_rendimiento(rf,  "Random Forest", X_train, y_train, cv)
evaluar_rendimiento(et,  "Extra Trees", X_train, y_train, cv)

Rendimiento de Árbol de decisión:	0.956 ± 0.008
Rendimiento de Bagging AD:	0.945 ± 0.014
Rendimiento de Random Forest:	0.938 ± 0.012
Rendimiento de Extra Trees:	0.945 ± 0.012


En este caso, el bagging de árboles de decisión anda mejor que el resto.   
Con otros set de datos, los modelos Random Forest y Extra Trees podrían tener mejores resultados y merecen ser probados. Podríamos implementar un gridsearh para intentar realizar un tunning de los hiperparámetros...

## 3. Tuneando los hiperparámetros de RandomForest

In [13]:
from sklearn.model_selection import GridSearchCV
param_trees = {'n_estimators': [50, 100, 200],
               'max_features': [1, 5, 8, 10, 21],
               'max_depth': [5, 20, 50, 70, 100],
               'min_samples_leaf':[1, 5, 8, 10, 50]}

In [15]:
grid_search_rf = GridSearchCV(rf, param_grid=param_trees, cv=cv, verbose=1, n_jobs=-1)

In [16]:
grid_search_rf.fit(X_train, y_train)

Fitting 3 folds for each of 375 candidates, totalling 1125 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=41, shuffle=True),
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=-1,
             param_grid={'max_depth': [5, 20, 50, 70, 100],
                         'max_features': [1, 5, 8, 10, 21],
                         'min_samples_leaf': [1, 5, 8, 10, 50],
                         'n_estimators': [50, 100, 200]},
             verbose=1)

In [17]:
grid_search_rf.best_score_

0.9520264681555005

In [21]:
mejor=grid_search_rf.best_estimator_
mejor

RandomForestClassifier(class_weight='balanced', max_depth=20, max_features=10,
                       n_estimators=200, random_state=1)

Puede verse que realizando un proceso de tunnig es ahora RandomForest el algoritmo que mejora la perfomance de los clasificadores comparados.

In [22]:
evaluar_rendimiento(mejor,  "Random Forest GS", X_train, y_train, cv)

Rendimiento de Random Forest GS:	0.952 ± 0.013
